Original https://ts.gluon.ai/stable/tutorials/forecasting/quick_start_tutorial.html

In [ ]:
!pip install --upgrade mxnet
#==1.6.0
!pip install gluonts
#!pip install pytorch-lightning
!pip install orjson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 4.4 MB/s eta 0:00:00


In [ ]:
import gluonts
import orjson
#from gluonts.torch.model.deepar import DeepAREstimator
from gluonts.mx.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
import numpy as np
import pandas as pd
from gluonts.dataset.common import ListDataset

In [ ]:
N = 10  # number of time series
T = 100  # number of timesteps
prediction_length = 24
freq = "1H"
custom_dataset = np.random.normal(size=(N, T))
start = pd.Period("01-01-2019", freq=freq)  # can be different for each time series
# train dataset: cut the last window of length "prediction_length", add "target" and "start" fields
train_ds = ListDataset(
    [{"target": x, "start": start} for x in custom_dataset[:, :-prediction_length]],
    freq=freq,
)
# test dataset: use the whole dataset, add "target" and "start" fields
test_ds = ListDataset(
    [{"target": x, "start": start} for x in custom_dataset], freq=freq
)
trainer = Trainer(epochs=20, learning_rate=1e-3)
# batch_size=512, patience=5
estimator = DeepAREstimator(freq=freq, context_length=24 * 5, ## context length is number of time steps will look back here it is 5 days
                            prediction_length=prediction_length, cardinality=[1],
                            num_layers=2, dropout_rate = 0.25)
#use_feat_static_cat=True, num_cells=32, cell_type='lstm', trainer=trainer
predictor = estimator.train(train_ds)

  2%|▏         | 1/50 [00:00<00:24,  1.97it/s, epoch=51/100, avg_epoch_loss=-.228]
/usr/local/lib/python3.10/dist-packages/gluonts/mx/trainer/_base.py:484: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(
